# t検定をスクラッチで実装する

t検定は **母平均** に対する検定。「グループA,Bの平均値が異なるかどうか」、「Aの平均値が0かどうか」など

## 1. データセット

suumoから千代田区と北区の賃料を取得する

## 2. Studentのt検定（等分散を仮定）

今回のデータは対応がない2群のt検定となる。ここでは等分散を仮定したt検定を実施

In [98]:
import numpy as np

np.random.seed(10)
m = 100
n = 120
a = np.random.normal(loc=50, scale=10, size=m)
b = np.random.normal(loc=48, scale=12, size=n)

### 2-1. 各種統計量を求める

In [103]:
# a 平均と不偏分散
a_mean = a.mean()
a_var = a.var(ddof=1)
# b 平均と不偏分散
b_mean = b.mean()
b_var = b.var(ddof=1)

In [104]:
# poolした分散
s_2 = ((m - 1) * a_var + (n - 1) * b_var) / (m + n - 2)
# 自由度
dof = m + n - 2

### 2-2. t値、p値を求める

In [106]:
from scipy.stats import t

t_value = (a_mean - b_mean) / (np.sqrt(s_2 * (1/m + (1/n))))

dof = m + n - 2
if t_value >= 0:
    p_value = (1 - t.cdf(t_value, dof)) * 2
else:
    p_value = t.cdf(t_value, dof) * 2

print(f't値：{t_value} p値：{p_value}')

t値：1.5542287653302505 p値：0.12158039393310838


### 2-3. scipyのt検定

In [107]:
stats.ttest_ind(a, b)

Ttest_indResult(statistic=1.5542287653302505, pvalue=0.12158039393310824)

値がほぼ一致していることが分かる。

## 3. Weltchのt検定（等分散を仮定しない）

### 3-1. 各種統計量を求める

In [109]:
# a 平均と不偏分散
a_mean = a.mean()
a_var = a.var(ddof=1)
# b 平均と不偏分散
b_mean = b.mean()
b_var = b.var(ddof=1)

In [118]:
# 使用する分散
s_2 = a_var / m + b_var / n
# 自由度
dof = int((a_var / m + b_var / n)**2 / (((a_var/m)**2 / (m-1)) + ((b_var/n)**2 / (n-1))))

### 3-2. t値、p値を求める

In [119]:
from scipy.stats import t

t_value = (a_mean - b_mean) / np.sqrt(s_2)

if t_value >= 0:
    p_value = (1 - t.cdf(t_value, dof)) * 2
else:
    p_value = t.cdf(t_value, dof) * 2

print(f't値：{t_value} p値：{p_value}')

t値：1.5832592076547947 p値：0.11481870230205526


### 3-3. scipyのt検定

In [120]:
stats.ttest_ind(a, b, equal_var=False)

Ttest_indResult(statistic=1.5832592076547947, pvalue=0.11481273851176753)

値がほぼ一致している